In [1]:
import org.apache.spark.sql.SparkSession

In [2]:
val spark = SparkSession.builder()
                        .master("yarn")
                        .appName("ch06_wiki_LSA-nsc")
                        .config("spark.jars","ch06-lsa-2.0.0-jar-with-dependencies.jar")
                        .getOrCreate()

spark = org.apache.spark.sql.SparkSession@5207476f


Spark application_1533622723243_13396: Some(http://gw02.itversity.com:4042)

In [9]:
spark.range(10000).repartition(10).toDF().show()

+----+
|  id|
+----+
|2675|
|2019|
|3284|
| 381|
|2596|
|3292|
|3382|
| 235|
|3338|
|2265|
|2975|
|2164|
|1834|
|4941|
|3129|
|4126|
|2847|
|3740|
|3426|
|2458|
+----+
only showing top 20 rows



In [ ]:
impor

In [10]:
import edu.umd.cloud9.collection.XMLInputFormat

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io._ //{LongWritable, Text}

Name: Unknown Error
Message: <console>:26: error: object umd is not a member of package edu
       import edu.umd.cloud9.collection.XMLInputFormat
                  ^

StackTrace: 

In [ ]:
import edu.umd.cloud9.collection.wikipedia._//WikipediaPage
import edu.umd.cloud9.collection.wikipedia.language._//EnglishWikipediaPage

In [ ]:
 def wikiXmlToPlainText(pageXml: String): Option[(String, String)] = {
    val page = new EnglishWikipediaPage()

    // Wikipedia has updated their dumps slightly since Cloud9 was written, so this hacky replacement is sometimes
    // required to get parsing to work.
    val hackedPageXml = pageXml.replaceFirst(
      "<text xml:space=\"preserve\" bytes=\"\\d+\">", "<text xml:space=\"preserve\">")

    WikipediaPage.readPage(page, hackedPageXml)
    if (page.isEmpty || !page.isArticle || page.isRedirect || page.isDisambiguation ||
        page.getTitle.contains("(disambiguation)")) {
      None
    } else {
      Some((page.getTitle, page.getContent))
    }
  }

In [ ]:
val path = "/user/kranthidr/dataSets/wikidump-02-sept-2018.xml"
@transient val conf = new Configuration()

    conf.set(XMLInputFormat.START_TAG_KEY, "<page>")
    conf.set(XMLInputFormat.END_TAG_KEY, "</page>")

    val kvs = spark.sparkContext.newAPIHadoopFile(path, 
                                                  classOf[XMLInputFormat], 
                                                  classOf[LongWritable],
                                                  classOf[Text], 
                                                  conf)

In [ ]:
    val rawXmls = kvs.map(_._2.toString).toDS()

In [ ]:
rawXmls.take(2)

In [ ]:
  val docTexts =  rawXmls.filter(_ != null).flatMap(wikiXmlToPlainText)

In [ ]:
docTexts.take(1).foreach(println)

In [ ]:
// docTexts.show(1, false) 
//Not Good print

In [ ]:
docTexts

In [ ]:
rawXmls.printSchema()

In [ ]:
docTexts.printSchema()

In [ ]:
import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer

import edu.stanford.nlp.ling.CoreAnnotations._ //{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline._ //{Annotation, StanfordCoreNLP}

import java.util.Properties

import org.apache.spark.ml.feature._ //{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql._ //{DataFrame, Dataset, SparkSession}

In [ ]:
 def createNLPPipeline(): StanfordCoreNLP = {
    val props = new Properties()
    props.put("annotators", "tokenize, ssplit, pos, lemma")
    new StanfordCoreNLP(props)
  }

In [ ]:
  def isOnlyLetters(str: String): Boolean = {
    str.forall(c => Character.isLetter(c))
  }

In [ ]:
  def plainTextToLemmas(text: String, stopWords: Set[String], pipeline: StanfordCoreNLP)
    : Seq[String] = {
    val doc = new Annotation(text)
    pipeline.annotate(doc)
    val lemmas = new ArrayBuffer[String]()
    val sentences = doc.get(classOf[SentencesAnnotation])
    for (sentence <- sentences.asScala;
         token <- sentence.get(classOf[TokensAnnotation]).asScala) {
      val lemma = token.get(classOf[LemmaAnnotation])
      if (lemma.length > 2 && !stopWords.contains(lemma) && isOnlyLetters(lemma)) {
        lemmas += lemma.toLowerCase
      }
    }
    lemmas
  }

In [5]:
val stopWords = scala.io.Source.fromFile("./stopwords.txt").getLines().toSet

stopWords = Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, ...


lastException: Throwable = null


Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, from, you've, they've, what's, wouldn't, both, could, its, under, which, you'd, an, be, here's, into, where, he'll, her, themselves, were, more, we'd, where's, they're, who's, between, aren't, ours, about, doesn't, how's, against, during, no, very, we, ha

In [6]:
val bStopWords = spark.sparkContext.broadcast(stopWords)

bStopWords = Broadcast(0)


Broadcast(0)

In [7]:
bStopWords.value

Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, from, you've, they've, what's, wouldn't, both, could, its, under, which, you'd, an, be, here's, into, where, he'll, her, themselves, were, more, we'd, where's, they're, who's, between, aren't, ours, about, doesn't, how's, against, during, no, very, we, ha

In [ ]:
val terms: Dataset[(String, Seq[String])] = 
docTexts.mapPartitions { iter =>
      val pipeline = createNLPPipeline()
      iter.map { case (title, contents) => (title, plainTextToLemmas(contents, bStopWords.value, pipeline)) }
    }

In [ ]:
terms.take(1)